In [1]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re

In [2]:
rech = 'https://rech.com/busca?q='

In [3]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [4]:
def findOEM(text):
    print(text)
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [5]:
def rechSearch(nav, cod, index):
    nav.get(str(rech + cod))
    try:
        elemento = nav.find_element(By.ID, 'showcase')
        elementos = elemento.find_elements(By.CLASS_NAME, 'y-item-wrapper-showcase')
        urls = []
        for elemento in elementos:
            url = elemento.find_element(By.TAG_NAME, 'link').get_attribute('href')
            urls.append(url)
            print(url)
        for url in urls:
            src = []
            keys = []
            values = []
            nav.get(str(url))
            nav.implicitly_wait(0.8)
            elementos = nav.find_elements(By.CLASS_NAME, 'zoomImg')
            for elemento in elementos:
                src.append(elemento.get_attribute('src'))
            nome = nav.find_element(By.CLASS_NAME, 'product__name').text
            codigo = nav.find_element(By.CLASS_NAME, 'product__partcode').text
            descricao = nav.find_element(By.CLASS_NAME, 'panel-description').text
            elemento = nav.find_element(By.CLASS_NAME, 'table-specifications')
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr th')
            for k in elementos:
                keys.append(k.text)
            elementos = elemento.find_elements(By.CSS_SELECTOR, '.product .product-details table tr td')
            for v in elementos:
                values.append(v.text)
            dict_infos = dict(zip(keys, values))
            for x in dict_infos.keys():
                y = dict_infos.get(x)
                y = y.replace('\n', ', ')
                dict_infos.update({x : y})
            temp = dict_infos['Aplicação Equipamentos'].split(' (')
            equips = []
            for item in temp:
                item = item.replace('1 ATÉ 999999999) -', '')
                item = item.replace(', VER MAIS', '')
                if item:
                    item = item.strip()
                    equips.append(item)
            temp = dict_infos['Código da Montadora'].split(',')
            cods = []
            for item in temp:
                cods.append(item.strip())
            print(nome)
            print(codigo)
            print(descricao)
            dict_infos.update({'Aplicação Equipamentos': equips})
            dict_infos.update({'Código da Montadora': cods})
            print(dict_infos)
            
    except Exception as e:
        print(e)
        pass

In [6]:
def loginRech(nav):
    nav.get('https://www.rech.com/login?targetUri=%2F')
    elemento = nav.find_element(By.ID, 'username')
    elemento.send_keys('yuri@cotripal.com.br')
    elemento = nav.find_element(By.ID, 'password')
    elemento.send_keys('Testes1!')
    elemento = nav.find_element(By.ID, 'submit')
    elemento.click()
    nav.implicitly_wait(0.8)

In [7]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)
# produtos = dict(zip(query['ID'], query['Produto']))

loginRech(nav)

In [8]:
rechSearch(nav, '87041572', 1)

https://www.rech.com/p/7511797/suporte-do-molinete-plante-h175727133-unitario
https://www.rech.com/p/7511798/suporte-do-molinete-plante-h175727001-unitario
Suporte do Molinete - Plante+ - H175727/133 - Unitário
Cód. H175727/133
SUPORTE MOLINETE
{'Marca': 'Plante+', 'Código do Produto': 'H175727/133', 'NCM': '84339090', 'Garantia': 'SEM GARANTIA ESTENDIDA', 'Código da Montadora': ['New Holland: 87041572', 'Case: 1347084C1', 'JOHN DEERE: H175727 H175727/G'], 'Aplicação Equipamentos': ['JOHN DEERE 1550CWS', 'JOHN DEERE 1550WTS', 'JOHN DEERE 600S', 'JOHN DEERE 614R', 'JOHN DEERE 625', 'JOHN DEERE 625D', 'JOHN DEERE 630', 'JOHN DEERE 630D', 'JOHN DEERE 630R', 'JOHN DEERE 635D', 'JOHN DEERE 635R', 'JOHN DEERE 640D', 'JOHN DEERE SÉRIE 300', 'JOHN DEERE SÉRIE 600', 'JOHN DEERE SÉRIE 600 - 616', 'JOHN DEERE SÉRIE 600 - 618', 'JOHN DEERE SÉRIE 600 - 620', 'JOHN DEERE SÉRIE 600 - 622', 'JOHN DEERE SÉRIE 600 - 635', 'JOHN DEERE SÉRIE 600 F', 'JOHN DEERE SÉRIE 600 R', 'JOHN DEERE SÉRIE 600FD - 630F